Notebook outlining what functions from nltk will be useful for pre-processing the text and training the classification algorithm.

Expects a CSV file in the current directory containing a release's date, text, and classification (pos or neg) in each row.

In [1]:
import pandas as pd
import nltk
import re

import sklearn

Some pre-processing to get the dataframe into the format we need for content analysis.

In [2]:
release_df = pd.read_csv('release_df.csv', index_col=0)

In [71]:
release_df.head()

,date,release_text,year,month
0,"January 31, 2018",Information received since the Federal Open Ma...,2018,January
1,"February 01, 2017",Information received since the Federal Open Ma...,2017,February
2,"March 15, 2017",Information received since the Federal Open Ma...,2017,March
3,"May 03, 2017",Information received since the Federal Open Ma...,2017,May
4,"June 14, 2017",Information received since the Federal Open Ma...,2017,June


In [15]:
release_df['year'] = release_df['date'].apply(lambda x: int(x[-4:]))

In [29]:
release_df['month'] = release_df['date'].apply(lambda x: re.findall(r'(\w+) \d', x)[0])

In [ ]:
release_df = release_df.reset_index(drop=True)

In [72]:
release_df.merge(ratesdf, how='left', on = ['year', 'month']).head()

,date,release_text,year,month,Effective Rate,Shadow Rate,Change,Positive (1) / Negative (-1)
0,"January 31, 2018",Information received since the Federal Open Ma...,2018,January,1.41,NaN,-0.08,-1.0
1,"February 01, 2017",Information received since the Federal Open Ma...,2017,February,0.66,NaN,-0.02,-1.0
2,"March 15, 2017",Information received since the Federal Open Ma...,2017,March,0.79,NaN,-0.2,-1.0
3,"May 03, 2017",Information received since the Federal Open Ma...,2017,May,0.91,NaN,-0.01,-1.0
4,"June 14, 2017",Information received since the Federal Open Ma...,2017,June,1.04,NaN,-0.14,-1.0


In [24]:
testrelease_df = release_df[(release_df['year']==2017) | (release_df['year']==2018)]

In [26]:
trainrelease_df = release_df.iloc[9:]

In [29]:
testrelease_df[:5]

,date,release_text,year,month
0,"January 31, 2018",Information received since the Federal Open Ma...,2018,January
1,"February 01, 2017",Information received since the Federal Open Ma...,2017,February
2,"March 15, 2017",Information received since the Federal Open Ma...,2017,March
3,"May 03, 2017",Information received since the Federal Open Ma...,2017,May
4,"June 14, 2017",Information received since the Federal Open Ma...,2017,June


In [28]:
trainrelease_df[:5]

,date,release_text,year,month
9,"January 27, 2016",Information received since the Federal Open Ma...,2016,January
10,"March 16, 2016",Information received since the Federal Open Ma...,2016,March
11,"April 27, 2016",Information received since the Federal Open Ma...,2016,April
12,"June 15, 2016",Information received since the Federal Open Ma...,2016,June
13,"July 27, 2016",Information received since the Federal Open Ma...,2016,July


In [42]:
rates2_df = pd.read_csv('rates2.7.csv')

In [44]:
rates2_df = rates2_df.rename(columns={"Month":'month', "Year":'year'})

In [53]:
rates2_df.drop(rates2_df.index[0], inplace=True)

In [54]:
rates2_df['month'] = rates2_df['month'].apply(lambda x: x.strip())

In [56]:
rates2_df['year'] = rates2_df['year'].apply(lambda x: int(x))

In [61]:
releaserates_df = rates2_df.merge(release_df, how='left', on = ['year', 'month'])

In [63]:
releaserates_df['Positive (1) / Negative (-1)'] = releaserates_df['Positive (1) / Negative (-1)'].apply(lambda x: int(x>0))

In [67]:
releaserates_df = releaserates_df.rename(columns={'Positive (1) / Negative (-1)' :'increase'})

In [69]:
releaserates_df.to_csv('releaserates.csv')

Now text processing using `nltk`.

In [75]:
releaserates_df = pd.read_csv('releaserates.csv', index_col=0)

In [78]:
releaserates_df.head()

,month,year,Effective Rate,Shadow Rate,Change,increase,date,release_text
0,January,2008,3.94,NaN,-24.37%,0,"January 22, 2008",The Federal Open Market Committee has decided ...
1,January,2008,3.94,NaN,-24.37%,0,"January 30, 2008",The Federal Open Market Committee decided toda...
2,February,2008,2.98,NaN,-12.42%,0,NaN,NaN
3,March,2008,2.61,NaN,-12.64%,0,"March 11, 2008",Since the coordinated actions taken in Decembe...
4,March,2008,2.61,NaN,-12.64%,0,"March 18, 2008",The Federal Open Market Committee decided toda...


In [ ]:
#release_df['tokenized'] = release_df['text']

In [31]:
releaseTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, min_df=3, stop_words='english', norm='l2')

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)